In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds

import numpy as np
import pandas as pd
import os
import PIL
import PIL.Image
import glob

import cv2

import matplotlib.pyplot as plt
from matplotlib import gridspec

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Num GPUs: 1


2022-09-23 17:49:29.565739: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 17:49:29.595047: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 17:49:29.595174: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
ds_train = image_dataset_from_directory(
'tiny_imagenet/train',
image_size=(64,64),
batch_size=32)

Found 100000 files belonging to 200 classes.


2022-09-23 17:49:34.402338: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 17:49:34.402489: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 17:49:34.402591: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 17:49:34.917665: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 17:49:34.917800: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [5]:
ds_val = image_dataset_from_directory(
'tiny_imagenet/val',
image_size=(64,64),
batch_size=32)

Found 10000 files belonging to 200 classes.


In [5]:
inputs = keras.Input(shape=(64,64,3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

for size in [32,64,128,256,512]:
    residual = x
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    residual = layers.Conv2D(size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])


x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(200, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [9]:
opt = tf.optimizers.Adam(learning_rate=0.0001)

In [7]:
model.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [8]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="tiny_imagenet_feature_extraction_without_data_augmentation.keras",
save_best_only=True,
monitor="val_loss")
]

In [9]:
history = model.fit(
ds_train,
epochs=40,
batch_size = 32,
validation_data=ds_val,
callbacks=callbacks)

Epoch 1/40


2022-09-22 16:45:51.919332: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8303
2022-09-22 16:45:53.149437: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3125/3125 [==============================] - 84s 26ms/step - loss: 5.0058 - accuracy: 0.0261 - val_loss: 4.7524 - val_accuracy: 0.0425
Epoch 2/40
3125/3125 [==============================] - 81s 26ms/step - loss: 4.5022 - accuracy: 0.0636 - val_loss: 4.2078 - val_accuracy: 0.1038
Epoch 3/40
3125/3125 [==============================] - 81s 26ms/step - loss: 4.2015 - accuracy: 0.0960 - val_loss: 4.1600 - val_accuracy: 0.1108
Epoch 4/40
3125/3125 [==============================] - 80s 26ms/step - loss: 3.9872 - accuracy: 0.1239 - val_loss: 3.7770 - val_accuracy: 0.1630
Epoch 5/40
3125/3125 [==============================] - 80s 26ms/step - loss: 3.8171 - accuracy: 0.1502 - val_loss: 3.7121 - val_accuracy: 0.1756
Epoch 6/40
3125/3125 [==============================] - 81s 26ms/step - loss: 3.6807 - accuracy: 0.1713 - val_loss: 3.7301 - val_accuracy: 0.1781
Epoch 7/40
3125/3125 [==============================] - 82s 26ms/step - loss: 3.5663 - accuracy: 0.1919 - val_loss: 3.4614 - val_accura

In [14]:
data_augmentation = keras.Sequential(
[
layers.RandomFlip("horizontal"),
layers.RandomRotation(0.1),
layers.RandomZoom(0.2),
]
)

In [15]:
inputs = keras.Input(shape=(64,64,3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

for size in [32,64,128,256,512]:
    residual = x
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    residual = layers.Conv2D(size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])


x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(200, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [16]:
model.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [17]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="tiny_imagenet_feature_extraction_with_data_augmentation.keras",
save_best_only=True,
monitor="val_loss")
]

In [18]:
history = model.fit(
ds_train,
epochs=40,
batch_size = 32,
validation_data=ds_val,
callbacks=callbacks)

Epoch 1/40
3125/3125 [==============================] - 103s 33ms/step - loss: 4.9569 - accuracy: 0.0296 - val_loss: 4.6860 - val_accuracy: 0.0517
Epoch 2/40
3125/3125 [==============================] - 102s 33ms/step - loss: 4.6017 - accuracy: 0.0596 - val_loss: 4.5876 - val_accuracy: 0.0652
Epoch 3/40
3125/3125 [==============================] - 102s 33ms/step - loss: 4.4018 - accuracy: 0.0816 - val_loss: 4.4666 - val_accuracy: 0.0808
Epoch 4/40
3125/3125 [==============================] - 103s 33ms/step - loss: 4.2397 - accuracy: 0.1018 - val_loss: 4.2962 - val_accuracy: 0.1023
Epoch 5/40
3125/3125 [==============================] - 103s 33ms/step - loss: 4.1074 - accuracy: 0.1210 - val_loss: 4.2338 - val_accuracy: 0.1086
Epoch 6/40
3125/3125 [==============================] - 100s 32ms/step - loss: 3.9953 - accuracy: 0.1359 - val_loss: 4.0009 - val_accuracy: 0.1347
Epoch 7/40
3125/3125 [==============================] - 98s 31ms/step - loss: 3.9024 - accuracy: 0.1505 - val_loss: 3.

In [6]:
model = keras.models.load_model("tiny_imagenet_feature_extraction_with_data_augmentation.keras")

In [7]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="tiny_imagenet_feature_extraction_with_data_augmentation_2.keras",
save_best_only=True,
monitor="val_loss")
]

In [8]:
model.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [9]:
history = model.fit(
ds_train,
epochs=40,
batch_size = 32,
validation_data=ds_val,
callbacks=callbacks)

Epoch 1/40


2022-09-22 19:37:36.985978: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8303
2022-09-22 19:37:38.193005: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3125/3125 [==============================] - 108s 33ms/step - loss: 2.9824 - accuracy: 0.3068 - val_loss: 3.2612 - val_accuracy: 0.2629
Epoch 2/40
3125/3125 [==============================] - 104s 33ms/step - loss: 2.9593 - accuracy: 0.3100 - val_loss: 3.3097 - val_accuracy: 0.2616
Epoch 3/40
3125/3125 [==============================] - 104s 33ms/step - loss: 2.9450 - accuracy: 0.3126 - val_loss: 3.3361 - val_accuracy: 0.2529
Epoch 4/40
3125/3125 [==============================] - 104s 33ms/step - loss: 2.9266 - accuracy: 0.3188 - val_loss: 3.2883 - val_accuracy: 0.2607
Epoch 5/40
3125/3125 [==============================] - 97s 31ms/step - loss: 2.9075 - accuracy: 0.3217 - val_loss: 3.3697 - val_accuracy: 0.2523
Epoch 6/40
3125/3125 [==============================] - 97s 31ms/step - loss: 2.9035 - accuracy: 0.3208 - val_loss: 3.2176 - val_accuracy: 0.2704
Epoch 7/40
3125/3125 [==============================] - 97s 31ms/step - loss: 2.8812 - accuracy: 0.3264 - val_loss: 3.1960 - val_ac

In [6]:
model = keras.models.load_model("tiny_imagenet_feature_extraction_with_data_augmentation_2.keras")

In [7]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="tiny_imagenet_feature_extraction_with_data_augmentation_3.keras",
save_best_only=True,
monitor="val_loss")
]

In [8]:
model.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [9]:
history = model.fit(
ds_train,
epochs=40,
batch_size = 32,
validation_data=ds_val,
callbacks=callbacks)

Epoch 1/40


2022-09-23 08:32:53.484739: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8303
2022-09-23 08:32:54.676369: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3125/3125 [==============================] - 109s 34ms/step - loss: 2.5397 - accuracy: 0.3904 - val_loss: 3.0177 - val_accuracy: 0.3186
Epoch 2/40
3125/3125 [==============================] - 104s 33ms/step - loss: 2.5329 - accuracy: 0.3922 - val_loss: 2.9921 - val_accuracy: 0.3226
Epoch 3/40
3125/3125 [==============================] - 103s 33ms/step - loss: 2.5147 - accuracy: 0.3964 - val_loss: 2.9239 - val_accuracy: 0.3344
Epoch 4/40
3125/3125 [==============================] - 97s 31ms/step - loss: 2.5127 - accuracy: 0.3959 - val_loss: 2.9733 - val_accuracy: 0.3244
Epoch 5/40
3125/3125 [==============================] - 97s 31ms/step - loss: 2.5033 - accuracy: 0.3972 - val_loss: 2.9906 - val_accuracy: 0.3273
Epoch 6/40
3125/3125 [==============================] - 96s 31ms/step - loss: 2.4983 - accuracy: 0.3984 - val_loss: 2.9339 - val_accuracy: 0.3290
Epoch 7/40
3125/3125 [==============================] - 97s 31ms/step - loss: 2.4880 - accuracy: 0.4015 - val_loss: 2.9809 - val_acc

In [7]:
model = keras.models.load_model("tiny_imagenet_feature_extraction_with_data_augmentation_3.keras")

In [8]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="tiny_imagenet_feature_extraction_with_data_augmentation_4.keras",
save_best_only=True,
monitor="val_loss")
]

In [9]:
model.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [10]:
history = model.fit(
ds_train,
epochs=40,
batch_size=32,
validation_data=ds_val,
callbacks=callbacks)

Epoch 1/40


2022-09-23 09:56:24.188832: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8303
2022-09-23 09:56:25.297220: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3125/3125 [==============================] - 140s 44ms/step - loss: 2.2922 - accuracy: 0.4401 - val_loss: 2.7462 - val_accuracy: 0.3751
Epoch 2/40
3125/3125 [==============================] - 133s 42ms/step - loss: 2.2879 - accuracy: 0.4390 - val_loss: 2.8665 - val_accuracy: 0.3572
Epoch 3/40
3125/3125 [==============================] - 134s 43ms/step - loss: 2.2862 - accuracy: 0.4417 - val_loss: 2.7249 - val_accuracy: 0.3735
Epoch 4/40
3125/3125 [==============================] - 134s 43ms/step - loss: 2.2812 - accuracy: 0.4412 - val_loss: 2.6694 - val_accuracy: 0.3838
Epoch 5/40
3125/3125 [==============================] - 143s 46ms/step - loss: 2.2795 - accuracy: 0.4410 - val_loss: 2.8783 - val_accuracy: 0.3520
Epoch 6/40
3125/3125 [==============================] - 107s 34ms/step - loss: 2.2611 - accuracy: 0.4445 - val_loss: 2.8425 - val_accuracy: 0.3581
Epoch 7/40
3125/3125 [==============================] - 100s 32ms/step - loss: 2.2565 - accuracy: 0.4468 - val_loss: 2.7201 - val

In [6]:
model = keras.models.load_model("tiny_imagenet_feature_extraction_with_data_augmentation_4.keras")

In [7]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="tiny_imagenet_feature_extraction_with_data_augmentation_5.keras",
save_best_only=True,
monitor="val_loss")
]

In [10]:
model.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [11]:
history = model.fit(
ds_train,
epochs=100,
batch_size=32,
validation_data=ds_val,
callbacks=callbacks)

Epoch 1/100


2022-09-23 17:50:07.141578: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8303
2022-09-23 17:50:08.350708: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3125/3125 [==============================] - 109s 34ms/step - loss: 2.1621 - accuracy: 0.4640 - val_loss: 2.7852 - val_accuracy: 0.3723
Epoch 2/100
3125/3125 [==============================] - 105s 34ms/step - loss: 2.1553 - accuracy: 0.4651 - val_loss: 2.7105 - val_accuracy: 0.3832
Epoch 3/100
3125/3125 [==============================] - 104s 33ms/step - loss: 2.1482 - accuracy: 0.4678 - val_loss: 2.7676 - val_accuracy: 0.3762
Epoch 4/100
3125/3125 [==============================] - 102s 33ms/step - loss: 2.1458 - accuracy: 0.4670 - val_loss: 2.7887 - val_accuracy: 0.3763
Epoch 5/100
3125/3125 [==============================] - 101s 32ms/step - loss: 2.1471 - accuracy: 0.4665 - val_loss: 2.7755 - val_accuracy: 0.3743
Epoch 6/100
3125/3125 [==============================] - 104s 33ms/step - loss: 2.1433 - accuracy: 0.4688 - val_loss: 2.7665 - val_accuracy: 0.3768
Epoch 7/100
3125/3125 [==============================] - 103s 33ms/step - loss: 2.1398 - accuracy: 0.4704 - val_loss: 2.8087

KeyboardInterrupt: 